In [ ]:

import numpy as np
#import keras
import tensorflow
import glob
#from tensorflow.keras.utils import np_utils
from PIL import Image
from sklearn.model_selection import StratifiedKFold



#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold

# -------------------------------------------------------------------------------------
#                        初期設定部
# -------------------------------------------------------------------------------------

# GrayScaleのときに1、COLORのときに3にする
COLOR_CHANNEL = 3

# 入力画像サイズ(画像サイズは正方形とする)
INPUT_IMAGE_SIZE = 224

# 訓練時のバッチサイズとエポック数
BATCH_SIZE = 32
EPOCH_NUM = 150

# 使用する訓練画像の入ったフォルダ(ルート)
TRAIN_PATH =  


# 使用する訓練画像の各クラスのフォルダ名
folder = ["100", "200", "250"]

# CLASS数を取得する
CLASS_NUM = len(folder)
print("クラス数 : " + str(CLASS_NUM))

#testを除いた中からvalidationの割合
validation_rate = 0.2

# 分割数を設定
k = 5

#分割交差検証の平均算出用
test_loss = [0]*k
test_accuracy = [0]*k
sum_accuracy = 0
sum_val_accuracy = 0
sum_test_accuracy = 0
sum_loss = 0
sum_val_loss = 0
sum_test_loss = 0

# -------------------------------------------------------------------------------------
#                        訓練画像入力部
# -------------------------------------------------------------------------------------

# 各フォルダの画像を読み込む
v_image = []
v_label = []
for index, name in enumerate(folder):
    dir = TRAIN_PATH + "\\" + name
    files = glob.glob(dir + "\\*.png")
    print(dir)
    for i, file in enumerate(files):
        if COLOR_CHANNEL == 1:
            img = load_img(file, color_mode = "grayscale", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        elif COLOR_CHANNEL == 3:
            img = load_img(file, color_mode = "rgb", target_size=(INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE))
        array = img_to_array(img)
        v_image.append(array)
        v_label.append(index)

v_image = np.array(v_image)
v_label = np.array(v_label)


# imageの画素値をint型からfloat型にする
v_image = v_image.astype('float32')
# 画素値を[0～255]⇒[0～1]とする
v_image = v_image / 255.0

# データの前処理
# ラベルを0から3の整数に変換
label_binarizer = LabelBinarizer()
#v_label = label_binarizer.fit_transform(v_label)

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

In [2]:

import sys
%matplotlib inline

import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#VGG16
input_shape=(224,224,3)
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=input_shape, name='block1_conv1'))
model.add(BatchNormalization(name='bn1'))
model.add(Activation('relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', name='block1_conv2'))
model.add(BatchNormalization(name='bn2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='block1_pool'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', name='block2_conv1'))
model.add(BatchNormalization(name='bn3'))
model.add(Activation('relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', name='block2_conv2'))
model.add(BatchNormalization(name='bn4'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='block2_pool'))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', name='block3_conv1'))
model.add(BatchNormalization(name='bn5'))
model.add(Activation('relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', name='block3_conv2'))
model.add(BatchNormalization(name='bn6'))
model.add(Activation('relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', name='block3_conv3'))
model.add(BatchNormalization(name='bn7'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='block3_pool'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block4_conv1'))
model.add(BatchNormalization(name='bn8'))
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block4_conv2'))
model.add(BatchNormalization(name='bn9'))
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block4_conv3'))
model.add(BatchNormalization(name='bn10'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='block4_pool'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block5_conv1'))
model.add(BatchNormalization(name='bn11'))
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block5_conv2'))
model.add(BatchNormalization(name='bn12'))
model.add(Activation('relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', name='block5_conv3'))
model.add(BatchNormalization(name='bn13'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='block5_pool'))
model.add(Flatten(name='flatten'))
model.add(Dense(units=4096, activation='relu', name='fc1'))
model.add(Dense(units=4096, activation='relu', name='fc2'))
model.add(Dense(units=CLASS_NUM, activation='softmax', name='predictions'))
#model.summary()



In [ ]:
for fold_num, (train_index, test_index) in enumerate(skf.split(v_image, v_label)):
    print(f"Fold {fold_num}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    # 訓練データとラベル
    train_data, train_labels = v_image[train_index], v_label[train_index]
    
    # テストデータとラベル
    test_data, test_labels = v_image[test_index], v_label[test_index]
    
    #labelbinarizer
    train_labels = label_binarizer.fit_transform(train_labels)
    test_labels = label_binarizer.fit_transform(test_labels)
    
    #シャッフル
    train_data, train_labels = shuffle(train_data, train_labels, random_state=0)
    
    # モデルのコンパイル
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    # 訓練
    history = model.fit(train_data, train_labels, validation_split = validation_rate, batch_size=BATCH_SIZE, epochs=EPOCH_NUM)
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    Accuracy = history.history['accuracy']
    val_Accuracy = history.history['val_accuracy']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.yscale('log')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, Accuracy, 'b', label='Training Accuracy')
    plt.plot(epochs, val_Accuracy, 'r', label='Validation Accuracy')
    plt.yscale('log')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    #print(Accuracy[-1])
    

    # 訓練データの予測
    test_predictions = model.predict(test_data)
    # 予測結果からクラスを選択
    test_predicted_classes = np.argmax(test_predictions, axis=1)
    # 正解ラベルをクラスに戻す
    test_true_classes = np.argmax(test_labels, axis=1)
    # 混同行列を計算
    confusion = confusion_matrix(test_true_classes, test_predicted_classes)
    # 混同行列を可視化
    display = ConfusionMatrixDisplay(confusion, display_labels=folder)
    display.plot(cmap=plt.cm.Blues, values_format="d")
    plt.title("Confusion Matrix (Training Data)")
    plt.show()
    
    score = model.evaluate(test_data, test_labels, verbose=0)
    print(len(test_data))
    test_loss[fold_num]=score[0]
    test_accuracy[fold_num]=score[1]
    print('Loss:', score[0])
    print('Accuracy:', score[1])
    
    sum_accuracy = sum_accuracy + Accuracy[-1]
    sum_val_accuracy = sum_val_accuracy + val_Accuracy[-1]
    sum_test_accuracy = sum_test_accuracy + score[1]
    sum_loss = sum_loss + loss[-1]
    sum_val_loss = sum_val_loss + val_loss[-1]
    sum_test_loss = sum_test_loss + score[0]
        
ave_accuracy = sum_accuracy/k
ave_val_accuracy = sum_val_accuracy/k
ave_test_accuracy = sum_test_accuracy/k
ave_loss = sum_loss/k
ave_val_loss = sum_val_loss/k
ave_test_loss = sum_test_loss/k

for j in range(k):
    print(f"hold{j+1}_test_loss:{test_loss[j]}")

    
for j in range(k):
    print(f"hold{j+1}_test_accuracy:{test_accuracy[j]}")
    

print(f"平均accuracy: {ave_accuracy}")
print(f"平均val_accuracy: {ave_val_accuracy}")
print(f"平均test_accuracy: {ave_test_accuracy}")
print(f"平均loss:{ave_loss}")
print(f"平均val_loss:{ave_val_loss}")
print(f"平均test_loss:{ave_test_loss}")

